In [1]:
from Uncertainty import *
from uncertaintyMetric import *
from utils import *

K.clear_session()
check_tensorflow_gpu()

Checking Tensorflow Version:
Tensorflow built with CUDA? True
Tensorflow version: 2.6.2
# GPU available: 1
CUDA: 64_112 | cuDNN: 64_8
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
result_presWAT = run_BruteForce_opt(measure_type=1, nMCSamples=5000, ROM_data=Earth(), ROM_obj=Earth())
result_co2sl   = run_BruteForce_opt(measure_type=2, nMCSamples=5000, ROM_data=Earth(), ROM_obj=Earth())
result_temp    = run_BruteForce_opt(measure_type=3, nMCSamples=5000, ROM_data=Earth(), ROM_obj=Earth())
reuslt_presWAT_co2sl = run_BruteForce_opt(measure_type=4, nMCSamples=5000, ROM_data=Earth(), ROM_obj=Earth())

### COLUMN/WELL OPTIMIZATION
well_names = ['well {}'.format(p+1) for p in range(16)]
templist, well = np.arange(2,50,step=16), {}
for i in range(16):
    well[i] = templist+i
wells = np.array(list(well.values()))
measures = ['presWAT', 'co2sl', 'temp']
result_arr = np.zeros((16,3))
for m in range(len(measures)):
    for w in range(wells.shape[0]):
        result_arr[w,m] = Proxy(ncol_data=list(wells[w]), measure_type=m+1, nMCSamples=5000, 
                                            rom_data=Earth(), rom_obj=Earth(), verbose=True).value
        print('column {} done'.format(w))
    print('measure {} done'.format(m))
print('... DONE ...')

results = pd.DataFrame(result_arr, columns=measures)
results.to_csv('_misael_results/optimization_by_column.csv', index=0)

pd.read_csv('_misael_results/optimization_by_column.csv')

c:\Users\381792\AppData\Local\anaconda3\envs\earth36\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
c:\Users\381792\AppData\Local\anaconda3\envs\earth36\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]
c:\Users\381792\AppData\Local\anaconda3\envs\earth36\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine prec

In [ ]:
result_presWat = np.load('_misael_results/result_presWAT_MARS.npy').reshape(3,16)
result_co2sl   = np.load('_misael_results/result_co2sl_MARS.npy').reshape(3,16)
result_temp    = np.load('_misael_results/result_temp_MARS.npy').reshape(3,16)
result_presWAT_co2sl = np.load('_misael_results/result_presWAT_co2sl_MARS.npy').reshape(3,16)
data = [result_presWat, result_co2sl, result_temp, result_presWAT_co2sl]
titles = ['presWat','co2sl','temp','presWAT_co2sl']

plt.figure(figsize=(20,17.5))
for i in range(4):
    plt.subplot(4,1,i+1)
    plt.imshow(data[i], cmap='gist_heat', vmin=-30, vmax=0)
    plt.title(titles[i]); plt.xlabel('Well Number'); plt.ylabel('Sensor Location')
    plt.yticks(np.arange(3), labels=np.arange(1,4))
    plt.colorbar(label='uncertainty reduction [$x10^6$]', fraction=0.046, pad=0.04)
plt.show()

In [ ]:
# select       = 7
# plot_bounds  = [-3, 3]

# local_method = 'trust-krylov'
# x0           = [1.5, 1.5]
# glob_bounds  = [-2, 2]

# opt = FullOpt(select)
# local_res,  local_res_df  = opt.local_opt(x0=x0, method=local_method)
# global_res, global_res_df = opt.global_opt(varbounds=glob_bounds)
# opt.make_plot(global_res_df, local_res_df, mbounds=plot_bounds, 
#               showcontours=True, showtrajectory=True, angle=[45,225])

Brute Force

In [ ]:
view_results('LR', suptitle='Linear Regression ROM', folder='_misael_results')
view_results('MARS', suptitle='MARS ROM', folder='_misael_results')

In [ ]:
plt.figure(figsize=(15,6))
plt.imshow(well_placement_results.T, aspect='auto', cmap='jet_r')
plt.xlabel('Well #'); plt.ylabel('Measure Type')
plt.title('Uncertainty Reduction by Well Location')
plt.yticks(np.arange(3), labels=labels)
plt.xticks(np.arange(16), labels=np.arange(1,17))
plt.colorbar(label='Uncertainty Reduction [$x10^6$]')
plt.show()

labels = ['presWAT','co2sl','temp']
well_placement_results = result_arr[:,:3]
plt.figure(figsize=(15,6))
plt.plot(well_placement_results[:,:3], '.-')
plt.legend(labels)
plt.xlabel('Well #'); plt.ylabel('Uncertainty Reduction [$x10^6$]')
plt.title('Uncertainty Reduction by Well Location')
plt.show()

***

In [ ]:
# General settings
Main_Directory   = os.getcwd()
Data_Directory   = os.path.join(Main_Directory, 'data')
titles           = ['.', 'Pressure', 'CO2 Saturation (l)', 'Temperature', 'Pressure + CO2 Saturation']
unc_params       = ['perm4', 'perm5', 'perm6', 'kmult']

MeasureType      = 1                           # Measurement type: 1 for pressure; 2 for CO2 saturation; 3 for tempeture, 4 for pressure+CO2 saturation                     
Total_time       = 1800                        # Total injection and post-injection time (days)
nTimeSeries      = 60                          # The number of measurement time series
nInterval        = 1                           # Measurement interval, default value = 1/Month

nTrain           = 500                         # The number of training simulations
Obj_filename     = 'run_co2mt.his'             # Objective file name
nColumn_obj      = 50                          # The column(s) in which the object of interest is located
nColumn_data     = [7]    #[7]                 # The column in which the data measurement is located 

x_max            = [1e-14, 1e-14, 1e-14, 2.0]  # Upper bounds for all the uncertain parameters
x_min            = [1e-19, 1e-19, 1e-19, 0.5]  # Lower bounds for all the uncertain parameters
ROMs_validation  = 0                           # ROMs cross-validation flag
nMCSamples       = 10000  #100000              # Number of monte carlo samples
nParam           = 4                           # Number of uncertain parameters
nDataRealization = 100    #200                 # Number of data realizations
err_option       = 3                           # Type of err option (2=MeanAE, 3=MaxAE)
time_sensitivity = 1                           # Whether consider time sensitivity for UR flag
post_processing  = 1                           # Plotting flag
print('Current Working Directory: {}'.format(Main_Directory))
print('Data Directory: {}'.format(Data_Directory))

## Step 1: Perfrom training simulation
# This step is done in a seperate code.

roms = [Ridge(), Ridge()]

***
# END